**notebook um aus einzelnen ergebnis json dateien für klassifikation und Maskengenerierung eine ergebnisdatei im coco format zu generieren und diese zu evaluieren mit cocoapi**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

#import utils
#import vision_transformer as vits

from pycocotools.coco import COCO
import json

In [ ]:
org_annotation = 'PATH TO ORIGINAL ANNOTATION FILE'

class_annotation_path = 'PATH TO KLASSIFIKATIONSERGEBNISSEN'

seg_annotation = 'PATH TO MASKENERGEBNISSEN'

In [ ]:
org_file = COCO(org_annotation)

with open(class_annotation_path) as json_file:
    class_annotation = json.load(json_file)

with open(seg_annotation) as file:
  segmentation_file = json.load(file)

In [ ]:
detections = []

for item in class_annotation:
  #print(item)
  image_id = int(item['image_id'])
  #print(image_id)
  bbox = item['bbox']
  category_id = int(item['mae_pred'])#int(item['category_id'])#   #anpassen zu knn_pred, und mlp_pred
  score = item['score']

  area_seg = [seg for seg in segmentation_file if seg['image_id'] == image_id]
  segmentation = area_seg[0]['pred_seg']

  segmentation['counts'] = segmentation['counts']
  area = area_seg[0]['pred_area']
  
  annotation = {}

  annotation['image_id'] = image_id
  annotation['bbox'] = bbox
  annotation['category_id'] = category_id
  annotation['score'] = score
  annotation['segmentation'] = segmentation


  detections.append(annotation)

*speichern der ergebnisjson in coco format*

In [ ]:
import json 

output_file = '/content/detection.json'

with open(output_file, 'w') as file:
  json.dump(detections, file, indent = 4, sort_keys=False)

In [ ]:
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab

In [ ]:
cocoGt = COCO(org_annotation)
cocoDt=cocoGt.loadRes('/content/detection.json')

In [ ]:
ann_type = ['segm','bbox']

imgIds=sorted(cocoGt.getImgIds())

cocoEval = COCOeval(cocoGt, cocoDt, ann_type[0])
#cocoEval.params.imgIds  = imgIds
#cocoEval.params.useCats = 0
#cocoEval.maxDets = [200]
#cocoEval.useCats = 1
#cocoEval.params.catIds = [4]
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()